# Wafer Defect Detection - Model Training
This notebook contains the complete training pipeline for the CNN model.

In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from collections import defaultdict

print("🎯 MANUAL DATA LOADING - No tf.data bugs")

## Data Loading

In [ ]:
# CORRECT PATH
DATASET_DIR = "/kaggle/input/data-v2"  # From your screenshot

# Find ALL class folders (recursive)
class_folders = defaultdict(list)
for root, dirs, files in os.walk(DATASET_DIR):
    if root != DATASET_DIR and len(files) > 0:
        rel_path = os.path.relpath(root, DATASET_DIR)
        class_name = os.path.basename(root)
        class_folders[class_name].extend([os.path.join(root, f) for f in files])

class_names = sorted(class_folders.keys())
print(f"✅ Found {len(class_names)} classes: {class_names}")

In [ ]:
# Load images manually
all_images = []
all_labels = []

for idx, class_name in enumerate(class_names):
    img_paths = class_folders[class_name][:500]  # Limit per class for memory
    print(f"Loading {class_name}: {len(img_paths)} images")

    for path in img_paths:
        img = tf.io.read_file(path)
        img = tf.image.decode_image(img, channels=1, expand_animations=False)
        img = tf.image.resize(img, [64, 64])
        img = tf.cast(img, tf.float32) / 255.0
        all_images.append(img)
        all_labels.append(idx)

print(f"\n✅ Total: {len(all_images)} images loaded")

In [ ]:
# Convert to numpy
X = np.array(all_images)  # [N, 64, 64, 1]
y = np.array(all_labels)

print(f"X shape: {X.shape}, y shape: {y.shape}")
print(f"Classes in data: {np.unique(y, return_counts=True)}")

In [ ]:
# Manual train/val split
split = int(0.8 * len(X))
X_train, X_val = X[:split], X[split:]
y_train, y_val = y[:split], y[split:]

print(f"Train: {X_train.shape}, Val: {X_val.shape}")

## Model Architecture

In [ ]:
# Model with CORRECT output classes
model = models.Sequential([
    layers.Input(shape=(64, 64, 1)),
    layers.Conv2D(16, 3, activation='relu', padding='same'),
    layers.Conv2D(16, 3, activation='relu', padding='same'),
    layers.Conv2D(32, 3, activation='relu', padding='same'),
    layers.Conv2D(32, 3, activation='relu', padding='same'),
    layers.GlobalAveragePooling2D(),
    layers.Dense(64, activation='relu'),
    layers.Dropout(0.3),
    layers.Dense(len(class_names), activation='softmax')  # FIXED
])

model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

print(model.summary())

## Training

In [ ]:
# TRAIN - Pure numpy arrays = NO BUGS
print("\n🚀 TRAINING (this WILL work)...")
history = model.fit(
    X_train, y_train,
    epochs=15,
    batch_size=16,
    validation_data=(X_val, y_val),  # TUPLE not list
    verbose=1
)

## Evaluation

In [ ]:
# Verify
print("\n✅ FINAL TEST:")
val_loss, val_acc = model.evaluate(X_val, y_val, verbose=0)
print(f"val_loss: {val_loss:.4f}, val_accuracy: {val_acc:.4f}")

## Export to TFLite

In [ ]:
# TFLite
print("\n🔧 TFLite...")
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_model = converter.convert()

with open('/kaggle/working/defect_model.tflite', 'wb') as f:
    f.write(tflite_model)

size = os.path.getsize('/kaggle/working/defect_model.tflite') / 1024
print(f"✅ TFLite: {size:.1f} KB - DOWNLOAD THIS!")